# Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd

# Visualização
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    LabelEncoder,
    StandardScaler,
    OneHotEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
     f1_score
)

# Algoritmos
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Carregar os dados

In [ ]:
# Carregar a base
df = pd.read_csv('/content/banco.csv')
df.head(5)

In [ ]:
# Informações
df.info()

In [ ]:
# Verificar NaN
df.isna().sum()

In [ ]:
# Criar colunas numéricas
colunas_numericas = df.select_dtypes(
               include =['int', 'float']).columns
colunas_numericas

In [ ]:
# Criar colunas categóricas
colunas_categoricas = df.select_dtypes(
                 include=['object']).columns
colunas_categoricas

# Funções auxiliares

In [ ]:
# Função para treinar
def treinar_avaliar(X_train, y_train, X_test, y_test):
    classificadores = {
        'Random Forest': RandomForestClassifier(random_state=42),
        'Knn': KNeighborsClassifier(),
        'Regressão Logística': LogisticRegression(random_state=42)
    }

    resultados = []

    for nome, classificador in classificadores.items():
        classificador.fit(X_train, y_train)
        y_pred = classificador.predict(X_test)
        acuracia = accuracy_score(y_test, y_pred)
        f1Score = f1_score(y_test, y_pred, average='weighted')
        resultados.append({'Modelo': nome,
                           'Acurácia': (acuracia*100).round(2),
                           'F1 Score': (f1Score*100).round(2)})

    return pd.DataFrame(resultados)

In [ ]:
# Função para a matriz de confusão
def plot_confusion_matrix(conf_matrix, labels):
    fig = ff.create_annotated_heatmap(z=conf_matrix[::-1],
                                      x=labels,
                                      y=labels[::-1],
                                      colorscale='blues')

    fig.update_layout(title='Matriz de Confusão',
                      xaxis=dict(title='Classe Prevista', side='bottom'),
                      yaxis=dict(title='Classe Real'))

    fig.show()

# EDA

In [ ]:
# Estatísticas Descritivas
df.describe().T.round(2)

In [ ]:
# Correlações
corr = df.corr(numeric_only=True)
corr.style.background_gradient()

In [ ]:
# Médias por alvo
pd.pivot_table(df, index='saiu',
               values=colunas_numericas,
               aggfunc={'mean', 'std'}).round(2)

In [ ]:
# Valores Únicos categóricos
for col in colunas_categoricas:
    print(col, df[col].unique())

In [ ]:
# Classe para gerar os gráficos
class Graficos:
    def __init__(self, df):
        self.df = df

    def histograma(self, col1):
        fig = px.histogram(self.df,
                           x=col1, marginal="box",
                           template='plotly_dark')
        fig.show()


    def boxplot_alvo(self, col1, col_genero='saiu'):
        fig = px.box(self.df,
                     x=col_genero, y=col1,
                     color = col_genero,
                     template='plotly_dark')

        fig.update_layout(xaxis_title='Saiu', yaxis_title=col1)

        fig.show()

    def barras(self, col1):
        # Contar ocorrências de valores na coluna
        counts = self.df[col1].value_counts()

        # Criar gráfico de barras
        fig = px.bar(x=counts.index,
                     y=counts.values,
                     text_auto = True,
                     template='plotly_dark')

        fig.update_traces(textposition='outside')

        fig.update_layout(xaxis_title=col1,
                          yaxis_title='Contagem',
                          )

        fig.show()

In [ ]:
# Criar o objeto
graficos = Graficos(df)

In [ ]:
# Histograma
for col in colunas_numericas:
    graficos.histograma(col)

In [ ]:
# Boxplot
for col in colunas_numericas:
    graficos.boxplot_alvo(col)

In [ ]:
# Barras
for col in colunas_categoricas:
    graficos.barras(col)

# Pré-processamento

In [ ]:
# Separar X e y
X = df.drop('saiu', axis = 1)
y = df['saiu']

In [ ]:
# Codificar alvo
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Armazenar os valores da classe
labels = list(le.classes_)

In [ ]:
# Definir colunas numéricas e categóricas
numeric_features = colunas_numericas
categorical_features = colunas_categoricas.drop('saiu')

In [ ]:
# Pré-processamento das colunas categóricas
categorical_transformer = OneHotEncoder(
                      handle_unknown='ignore')

In [ ]:
# Pipeline para pré-processamento das features numéricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [ ]:
# Combinar os pré-processadores em um ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ], remainder='drop')

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, test_size=0.2,
                                                    random_state=42,
                                                    stratify = y
                                                )

In [ ]:
# Aplicar o pré-processamento nos conjuntos de treino e teste
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Classificação

In [ ]:
# Treinar
resultados_df = treinar_avaliar(X_train_processed,
                             y_train,
                             X_test_processed,
                             y_test)

resultados_df

## Random Forest

In [ ]:
# Treinar
rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train_processed, y_train)

In [ ]:
# Avaliar
rf_acc = accuracy_score(y_test, rf.predict(X_test_processed))

print(f"Acurácia (treino):"
      f"{accuracy_score(y_train, rf.predict(X_train_processed)):.2f}")

print(f"Acurácia (teste): {rf_acc:.2f}")

In [ ]:
# Matriz de confusão
rf_matriz = confusion_matrix(y_test, rf.predict(X_test_processed))

# Mostrar matriz
plot_confusion_matrix(rf_matriz, labels)

In [ ]:
# Relatório de classificação
print(classification_report(y_test, rf.predict(X_test_processed)))